### 🤖 Hand Pose Estimation - 모델 학습 (Training)

이 노트북에서는 손 자세를 인식하는 AI 모델을 학습시킵니다.

#### 📚 학습 목표
- YOLO 모델을 사용하여 손의 keypoint(관절점)를 찾는 모델 만들기
- 하이퍼파라미터를 조정하며 모델 성능 비교하기
- 학습 과정을 모니터링하고 결과 분석하기

---

#### 1️⃣  필요한 라이브러리 설치 및 불러오기

In [ ]:
# YOLO 라이브러리 설치 (최초 1회만 실행)
!pip install ultralytics

In [ ]:
# 필요한 라이브러리 불러오기
import os
import warnings
warnings.filterwarnings('ignore')
import os
import yaml
import glob
from ultralytics import YOLO
from ultralytics import settings

print("✅ 라이브러리 로딩 완료!")

---
#### 2️⃣ 경로 설정

In [ ]:
# 현재 작업 디렉토리 확인
current_dir = os.getcwd()
new_runs_dir = os.path.abspath(os.path.join(current_dir, '../runs'))
new_data_dir = os.path.abspath(os.path.join(current_dir, '../data'))
# YOLO 설정 업데이트
settings.update({"runs_dir": new_runs_dir})
settings.update({"datasets_dir": new_data_dir})
settings.update({"wandb": False})  # wandb 로깅 비활성화

print("\n✅ 경로 설정 완료!")

---

#### 3️⃣ 데이터셋 규모 확인

모델을 학습시키기 전, **준비된 데이터가 총 몇 장인지** 확인하는 과정은 필수입니다.

> 원래 이 AI 모델을 제대로 학습시키려면 `130,000장 이상의 데이터(FreiHAND Full)`가 필요합니다.<br>
> 하지만 제한된 실습 시간 내에 학습과 결과를 모두 확인하기 위해, 우리는 전체 데이터의 `약 10% (Mini Subset)` 만을 사용하여 **빠르게 학습**을 진행하겠습니다.

- `DATA_CONFIG` : 데이터셋의 경로와 클래스 정보가 담긴 설정 파일(.yaml)의 위치

In [ ]:

DATA_CONFIG = "../data/config.yaml"


with open(DATA_CONFIG, 'r') as f:
    data = yaml.safe_load(f)

print(f"📘 dataset config 경로 : {DATA_CONFIG}")
print("-" * 30)

# 2. 데이터셋 기본 경로 확인 (path 키가 있으면 사용, 없으면 현재 경로 기준)
base_path = data.get('path', '')

# 3. 각 데이터셋(학습/검증/테스트) 별 이미지 개수 세기
for split in ['train', 'val', 'test']:
    if split in data:
        folder_path = os.path.join(base_path, data[split])
        if not os.path.isabs(folder_path):
            config_dir = os.path.dirname(os.path.abspath(DATA_CONFIG))
            pass
        image_count = 0
        for ext in ['*.jpg', '*.jpeg', '*.png']:
            image_count += len(glob.glob(os.path.join(folder_path, ext)))
        print(f"📂 {split.upper():<8} 데이터: {image_count:>5}장 \t(위치: {data[split]})")

print("-" * 30)

---
#### 4️⃣ 하이퍼파라미터 설정 🎯
**여기서 값을 변경하여 실험해보세요!**

##### 주요 하이퍼파라미터 설명:

- **MODEL_SIZE**: 모델 크기 선택
  - `'yolo11n-pose.pt'`: Nano (가장 빠름, 정확도 낮음)
  - `'yolo11s-pose.pt'`: Small (빠름, 정확도 보통)
  - `'yolo11m-pose.pt'`: Medium (보통, 정확도 높음)
  - `'yolo11l-pose.pt'`: Large (느림, 정확도 매우 높음)

- **EPOCHS**: 전체 데이터셋을 몇 번 반복 학습할지 (기본: 100)
  - 너무 적으면: 학습 부족 (underfitting)
  - 너무 많으면: 과적합 (overfitting) 위험

- **IMG_SIZE**: 입력 이미지 크기 (기본: 640)
  - 작을수록: 빠른 학습, 낮은 정확도
  - 클수록: 느린 학습, 높은 정확도

In [ ]:
# 모델 크기 선택 (n, s, m, l 중 선택)
MODEL_SIZE = 'yolo11n-pose.pt'  # 실험해보기: 'yolo11s-pose.pt', 'yolo11m-pose.pt'

# 학습 epoch 수
EPOCHS = 100  # 실험해보기: 50, 150, 200

# 이미지 크기
IMG_SIZE = 640  # 실험해보기: 416, 512, 800

print("📋 현재 설정:")
print(f"  - 모델 크기: {MODEL_SIZE}")
print(f"  - Epochs: {EPOCHS}")
print(f"  - 이미지 크기: {IMG_SIZE}")

---

#### 5️⃣ 모델 불러오기

In [ ]:
# YOLO Pose 모델 로드
model = YOLO(MODEL_SIZE)

print(f"\n✅ {MODEL_SIZE} 모델 로딩 완료!")
print(f"   모델 유형: Pose Estimation")
print(f"   파라미터 수: {sum(p.numel() for p in model.model.parameters()):,}")

---
#### 6️⃣ 모델 학습 시작! 🚀

이제 설정한 하이퍼파라미터로 모델을 학습합니다.

**학습 중 확인할 수 있는 정보:**
- `box_loss`: 손의 위치를 얼마나 정확히 찾는지 (낮을수록 좋음)
- `pose_loss`: keypoint(관절점) 위치를 얼마나 정확히 찾는지 (낮을수록 좋음)
- `kobj_loss`: keypoint가 있는지 없는지 판단 정확도 (낮을수록 좋음)
- `cls_loss`: 분류 정확도 (낮을수록 좋음)

**💡 팁:** 학습이 진행되면서 loss 값들이 감소하는지 확인하세요!

In [ ]:
print("🏋️ 모델 학습을 시작합니다...")
print("⏰ 시간이 좀 걸릴 수 있으니 기다려주세요!\n")

# 모델 학습
results = model.train(data=DATA_CONFIG, epochs=EPOCHS, imgsz=IMG_SIZE)

print("\n🎉 학습 완료!")

---
#### 7️⃣ 학습 결과 확인 📊

학습이 완료되면 다음 위치에 결과가 저장됩니다:
- `../runs/pose/train/`: 학습 결과 폴더
  - `weights/best.pt`: 가장 성능이 좋은 모델
  - `weights/last.pt`: 마지막 epoch의 모델
  - `results.png`: 학습 과정 그래프
  - `confusion_matrix.png`: 혼동 행렬
  - 기타 검증 이미지들

In [ ]:
# 학습 결과 경로 출력
save_dir = model.trainer.save_dir
print(f"📁 학습 결과 저장 위치: {save_dir}")
print(f"\n🏆 Best 모델: {save_dir}/weights/best.pt")
print(f"📦 Last 모델: {save_dir}/weights/last.pt")

# 주요 결과 파일 확인
result_files = [
    'results.png',
    'confusion_matrix.png',
    'val_batch0_pred.jpg',
]

print("\n📊 생성된 결과 파일들:")
for file in result_files:
    file_path = os.path.join(save_dir, file)
    if os.path.exists(file_path):
        print(f"  ✓ {file}")
    else:
        print(f"  ✗ {file} (없음)")